### Ferdinand Hoske
# Spotify Million Playlist Challenge

For tis project  I decided to work on the Million Playlist Challenge held by Spotify in 2018. Spotify made the dataset public in September 2020. The goal of the challenge is to recommend songs that suit a specific playlist. The so called automatic playlist continuation.

In [5]:
!pip install tekore lightfm
from google.colab import drive
import pandas as pd
import numpy as np
from scipy import sparse as sp
import json
import glob
import tekore as tk
import progressbar as pb
from lightfm import LightFM
from lightfm.data import Dataset

## 1. Datapreprocessing and Feature Engineering
The original dataset holds 1.000.000 playlists and over 2 Million unique tracks. For performance reasons we will work with a portion.

In [6]:
drive.mount("/content/drive")
files = glob.glob("/content/drive/My Drive/spotify playlist challenge/unzip/data/frac/*")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We first load the JSON files holding the playlists and concat them in a DataFrame called `data_playlists`

In [7]:
data_playlists = pd.DataFrame()
for file in files:
  single_json = json.load(open(file))
  df = pd.DataFrame(single_json["playlists"])
  data_playlists = data_playlists.append(df, ignore_index=True)

The Songs are saved as dict-like object inside the "tracks" column of each playlist. We want to have two additional DataFrames. One holding the interaction between playlist and track called `tracks_playlist`
 and one that holds each unique track with additional information called `data_tracks`

In [8]:
tracks_playlist = pd.DataFrame()
data_tracks = pd.DataFrame()

#iterates through playlist and saves songs in DataFrame
for i, playlist in data_playlists.iterrows():
  tracks_df = pd.DataFrame(playlist["tracks"])
  tracks_df["playlist_id"] = playlist["pid"]
  tracks_playlist = tracks_playlist.append(tracks_df, ignore_index=True)

#creates a new DataFrame that holds information of unique songs
data_tracks = tracks_playlist[tracks_playlist["track_uri"].duplicated() != True]

#change uri to id and drop columns that are not neccessary for DataFrame
data_tracks["id"] = data_tracks["track_uri"].str.lstrip("spotify:track:").values
tracks_playlist["track_id"] = tracks_playlist["track_uri"].str.lstrip("spotify:track:").values
data_playlists = data_playlists.drop("tracks", 1)
data_tracks = data_tracks.drop(["playlist_id", "pos", "track_uri"], 1)
tracks_playlist = tracks_playlist.drop(
    ["artist_name", "artist_uri", "track_name", "album_uri", "duration_ms", "album_name", "track_uri"], 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Additional to the data provided by the dataset we want to use the Spotify API to enrich the track information with **Audio Features**. The package tekore provides the methods to access the HTTP routes.

In [9]:
features = ["danceability", 
            "energy", 
            "speechiness", 
            "acousticness", 
            "instrumentalness", 
            "liveness", 
            "tempo", 
            "valence"]

app_token = tk.request_client_token("76e5eea6048a40b3b12da575781777a0", "4e294eb734dc462c992028985b40c930")
spotify = tk.Spotify(token=app_token, sender=tk.RetryingSender(retries=3))

tracks_features = pd.DataFrame()

#Access API with 100 track ids which is the maximum amount per call
i = 0
while i<len(data_tracks["id"].values):
  j = i + 100
  sp_features = spotify.tracks_audio_features(data_tracks["id"].values[i:j])
  tracks_features = tracks_features.append(sp_features)
  i = j

data_tracks[features] = tracks_features[features].values

#normalize tempo so it fits the other features scale from 0 to 1
data_tracks["tempo"] = (data_tracks["tempo"].values - data_tracks["tempo"].min())/(data_tracks["tempo"].max() - data_tracks["tempo"].min())

## 2. Model Evaluation Method
The challenge was evaluated based on different metrics  defined by Spotify which we also will use to evaluate or model.
### R-Precision
R-Precision is the number of predicted songs that match the ground truth devided by all songs in the playlist and averaged over all playlists.

$\text{R-precision}=\frac{|G\cap R|}{|R|}$

In [10]:
def get_r_precision(recommendations, playlist_track_list):
  playlist_length = playlist_track_list.size
  match = np.count_nonzero(np.in1d(recommendations, playlist_track_list))
  return match/playlist_length

We won't score the Normalized Discounted Cumulative Gain because we are missing the weights to calculate this metric. We also won't consider artist match becasue we are missing how they are weighted either.

## 3. Simple Recommender Model
The first approach is going to be a purely collaborative filtering method where we check for correlation between a song and each song in the playlist we want to recommend for. To calculate the correlation we use the Phi-Coefficient.

$\phi=\frac{n_{11}n_{00}-n_{10}n_{01}}{\sqrt{n_{\bullet 1}n_{1\bullet}n_{\bullet 0}n_{0\bullet}}}$

$n_{11}$ is the number of times the songs appear in the same playlist, $n_{00}$ how often they both dont appear in a playlist, $n_{10}$ the number of times the first song appears in a playlist the other isnt and $n_{01}$ vice versa.

In [11]:
def phi(a, b):
  a = M.loc[a].values
  b = M.loc[b].values
  n11 = len(np.where((a==1) & (b==1))[0])
  n10 = len(np.where((a==1) & (b==0))[0])
  n01 = len(np.where((a==0) & (b==1))[0])
  n00 = len(np.where((a==0) & (b==0))[0])
  return (n11*n00-n10*n01)/np.sqrt((n11+n01)*(n01+n00)*(n11+n10)*(n10+n00))

To feed the function we need a pivot table where the tracks are the rows and the playlists the columns and a 1 representing an appearance and a 0 an absence.

In [12]:
M = tracks_playlist.pivot_table(columns=["playlist_id"], index=["track_id"])
M.iloc[~M.isna()] = 1
M = M.fillna(0)

To get recommendations we calculate the phi coefficient of each track in our `data_tracks` DataFrame that is not included in our playlist and return the `track_id` in the order of the highest score.

In [13]:
def get_phi_recommendation(playlist_id):
  playlist_tracks = tracks_playlist["track_id"].values[tracks_playlist["playlist_id"] == playlist_id]

  func = lambda track: [track, np.sum([phi(x, track) for x in playlist_tracks if track not in playlist_tracks])/len(playlist_tracks)]
  recommendations = list(map(func, tracks_playlist["track_id"].unique()))

  recommendations.sort(key=lambda k: k[1], reverse= True)
  return recommendations

In [ ]:
import time
start = time.time()
recommendations = get_phi_recommendation(45)
print(time.time()-start)
recommendations[:10]

By self-testing the recommended songs the recommender gave good results but had two major downsides:
1. It was a lot of computing necessary.
2. You couldn't really score the results others than subjectively rate the recommendations because you obvsiously the songs of the playlist would always have the highest correlation so there is no ground truth for what to predict.

## 4. Simple Matrix Factorization Model
 The second approach is the Matrix Factorization. Similar to the concept of Single Value Decomposition are we aiming to factorize two matrices P and Q that hold latent features of our playlists and tracks.

We use stoachstic gradient descent to minimize the L2 loss for every element of P and Q


In [14]:
def matrix_factorization(M, K=5, steps=100, alpha=0.02, beta=0.02): 
  P = np.random.rand(M.shape[0],K)
  Q = np.random.rand(M.shape[1],K).T
  for step in pb.progressbar(range(steps)):
    for i in range(len(M)):
      for j in range(len(M[i])):
        if M[i][j] > 0:
          eij = M[i][j] - np.dot(P[i,:],Q[:,j])
          for k in range(K):
            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
            Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
    l = 0
    for i in range(len(M)):
      for j in range(len(M[i])):
        if M[i][j] > 0:
          l = l + pow(M[i][j] - np.dot(P[i,:],Q[:,j]), 2)
          for k in range(K):
            l = l + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
    if l < 0.001:
      break
  return P, Q

The dot product of the row or column of P and Q then represent how good a song would suit the playlist.

In [ ]:
P_hat, Q_hat = matrix_factorization(M.values)

pid = 45
playlist_track_list = tracks_playlist["track_id"].values[tracks_playlist["playlist_id"] == pid]
recommendations = M.index[np.dot(P_hat, Q_hat[:,pid]).argsort()[::-1]][:500]
print("R-Precision for simple Matrix Multiplication: ", get_r_precision(recommendations, playlist_track_list))

The results ware rather unsatisfying which could be due to the sparseness of the matrix, the binary of the data or the choice of loss function. The computation also took a lot of time. We further couldn't use this model for unseen data.

## 5. Hybrid Recommender Model
For our final approach we will use the popular package lightFM to build a Matrix Factorization model but enrich it with the audio features we gathered from the Spotify API to therefore build a hybrid collaborative and content based filtering model. 

The predict method of lightFM allows us to also work with unseen data. We therefore split our tracks in training and test data. The interaction table to train the data is basically the same as the pivot table but we use the `build_interactions` method for easier indexing capabilities.

In [15]:
#set train test split
train_split = 0.75
number_train_playlists = int(len(data_playlists)*train_split)
train_ids = np.arange(number_train_playlists)
test_ids = np.arange(number_train_playlists, len(data_playlists))

In [16]:
dataset= Dataset()
dataset.fit(
    users = train_ids, 
    items = tracks_playlist["track_id"].unique(),
    item_features = features
)
train, weights = dataset.build_interactions([(x[1], x[2]) for x in tracks_playlist.values[tracks_playlist["playlist_id"] < number_train_playlists]])

# Save track_features in lightfm specific format [(item_id, {feature1: value1, ...}), ...]
track_tuple= list(zip(data_tracks["id"], data_tracks[features].to_dict('records')))
item_features = dataset.build_item_features(track_tuple, normalize=False)

user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
item_id_map_rev = {value:key for key, value in item_id_map.items()}

For the challenge Spotify provided a challenge dataset with incomplete playlists to validate the model. These playlists follow the following structure
* first track
* first 5 tracks
* first 10 tracks
* first 25 tracks
* first 100 tracks

plus the combination with a missing title. For this model we are focusing on predicitng based on tracks. The complete dataset contains over 2 million songs. To have an overlapping of songs and the ground truth information we are not using the challenge dataset but build interactions based on the given structure.


In [17]:
test_set = np.empty(3)
for id in test_ids:
  playlist_track_list = tracks_playlist.values[tracks_playlist["playlist_id"] == id]

  if len(playlist_track_list)>=200:
    test_set = np.vstack([test_set, playlist_track_list[:100]])
  elif len(playlist_track_list)>=100:
    test_set = np.vstack([test_set, playlist_track_list[:25]])
  elif len(playlist_track_list)>=50:
    test_set = np.vstack([test_set, playlist_track_list[:10]])
  elif len(playlist_track_list)>=25:
    test_set = np.vstack([test_set, playlist_track_list[:5]])
  elif len(playlist_track_list)<25:
    test_set = np.vstack([test_set, playlist_track_list[:1]])

test_set = np.delete(test_set, (0), axis=0)

test_dataset = Dataset()
test_dataset.fit(
    users = test_ids, 
    items = tracks_playlist["track_id"].unique(),
    item_features = features
)
test, weights = test_dataset.build_interactions(
    [(x[1], x[2]) for x in test_set])

test_user_id_map, test_user_feature_map, test_item_id_map, test_item_feature_map = test_dataset.mapping()

To train our model we use 200 components to describe playlists and songs, a slightly slower learning rate, an initial state of one for creating the random parameters and the **Weighted Approximate-Rank Pairwise** loss. This has been shown to be useful for recommendation models and was also used by some of the participants in the 2018 challenge. To speed up training we set the maximum number of negative samples to 200.

In [18]:
model = LightFM(
    no_components=200, 
    loss='warp', 
    learning_rate=0.02, 
    max_sampled=400, 
    random_state=1, 
    user_alpha=1e-05)

model.fit(
    interactions=train, 
    item_features=item_features, 
    epochs=20, 
    num_threads=2)

In [19]:
def get_lightfm_recommendation(item):
  predicitions = model.predict(
      user_ids=0, 
      item_ids=np.arange(data_tracks.shape[0]), 
      item_features=item, 
      num_threads=2
  ).argsort()[::-1]
  return [item_id_map_rev[predicitions[i]] for i in range(500)]

In [22]:
r_values = []

for pid in train_ids:
  fm_id = user_id_map[pid]
  item = train.getrow(fm_id).transpose()
  playlist_track_list = tracks_playlist["track_id"].values[tracks_playlist["playlist_id"] == pid]

  recommendations = np.array(get_lightfm_recommendation(item))
  r_values.append(get_r_precision(recommendations, playlist_track_list))

print("R-Precision for Traindata: ", np.sum(r_values)/len(train_ids))


R-Precision for Traindata:  0.990830198148823


The final step is to calculate the R-Precision for our test_data

In [20]:
r_values = []

for pid in test_ids:
  fm_id = test_user_id_map[pid]
  item = test.getrow(fm_id).transpose()
  playlist_track_list = tracks_playlist["track_id"].values[tracks_playlist["playlist_id"] == pid]

  recommendations = np.array(get_lightfm_recommendation(item))
  r_values.append(get_r_precision(recommendations, playlist_track_list))

print("R-Precision for Testdata: ", np.sum(r_values)/len(test_ids))


R-Precision for Testdata:  0.16036571589559334


## 6. Conclusion and Outlook
During the challenge the top teams could reach a R-Precision between 0.22 and 0.2. If we compare them with our results of 0.16 our model did not excel but is not far of the leading research results either. It is to mention that during the challenge the score was also evaluated on playlists without tracks and only the title where the average was abut 0.09.

Most participants also worked with a Two stage model where a neural network reranked the results of the matrix factorization. A second stage therefore could also enhance our model.